In [1]:
#!/usr/bin/env python
import rospy
from human_moveit_config.human_model import HumanModel
import json
import rospkg
import time
from sensor_msgs.msg import JointState
from os.path import join
from os.path import exists
from baxter_commander.persistence import dicttostate
import sys
from reba_optim.reba_assess import RebaAssess
from os import listdir
from os.path import isdir
from numpy import mean
import numpy as np
import pylab as plt
import os
import shutil
from scipy.stats.mstats import kruskalwallis
from scipy.stats import wilcoxon
from scipy.stats import ttest_rel 
from scipy.stats import sem
from scipy.stats import mannwhitneyu
from itertools import permutations
from itertools import combinations

In [2]:
rospy.init_node('analyse_posture')
rospack = rospkg.RosPack()

In [3]:
reba_joints = ['neck_0', 'neck_1', 'neck_2', 'right_elbow_0', 'right_elbow_1',
               'right_shoulder_0', 'right_shoulder_1', 'right_shoulder_2', 
               'right_wrist_0', 'right_wrist_1', 'spine_0', 'spine_1', 'spine_2']

In [4]:
res_folder = join('results', 'user_study')

In [5]:
def plot_reba(data, savefile):
    width = 0.25
    ind = np.arange(6) + 1
    labels = ['neck', 'spine', 'shoulder', 'elbow', 'wrist', 'total']

    # extract data
    mean = {}
    std = {}
    for c in conditions:
        mean[c] = []
        std[c] = []
        for l in labels:
            if l in data[c].keys():
                mean[c].append(np.mean(data[c][l]))
                std[c].append(sem(data[c][l]))
            else:
                mean[c].append(np.mean(data[c]['right_' + l]))
                std[c].append(sem(data[c]['right_' + l]))
    
    fig = plt.figure(facecolor="white")
    ax = fig.add_subplot(111)
    plt.rcParams['font.size'] = 20

    rects1 = ax.bar(ind - 3 * width / 2, mean['fixed'], width, color='tomato', label='fixed')
    (line, caps, _) = plt.errorbar(ind - width, mean['fixed'], std['fixed'], capsize=3, elinewidth=2,  ecolor='k')

    line.remove()
    for cap in caps:
        cap.set_color('k')
        cap.set_markeredgewidth(3)

    rects2 = ax.bar(ind - width / 2, mean['relative'], width, color='darkseagreen', label='relative')
    (line, caps, _) = plt.errorbar(ind, mean['relative'], std['relative'], capsize=3, elinewidth=2,  ecolor='k')
    
    line.remove()
    for cap in caps:
        cap.set_color('k')
        cap.set_markeredgewidth(3)

    rects3 = ax.bar(ind + width / 2, mean['reba'], width, color='cornflowerblue', label='optimized')
    (line, caps, _) = plt.errorbar(ind + width, mean['reba'], std['reba'], capsize=3, elinewidth=2,  ecolor='k')

    line.remove()
    for cap in caps:
        cap.set_color('k')
        cap.set_markeredgewidth(3)
    
    legend = ax.legend(loc='upper left')
    # Set the fontsize
    for label in legend.get_texts():
        label.set_fontsize('small')

    for label in legend.get_lines():
        label.set_linewidth(1.5)  # the legend line width

    # add some text for labels, title and axes ticks
    ax.set_ylabel('Average REBA Score')
    ax.set_ylim([0, 5])
    y_tick = np.arange(5)
    y_label = [str(x) for x in y_tick]
    plt.yticks(y_tick, y_label, rotation='vertical')
    
    plt.xticks(ind, labels, rotation='vertical')
    ax.set_xlim([0, 7])

    # save svg file
    fig.set_size_inches(12.8, 10.24)
    plt.savefig(savefile, dpi=100, facecolor=fig.get_facecolor(), transparent=False)
    plt.close()

In [6]:
def plot_results(data, start, end, savefile):
    width = 0.25
    ind = np.arange(len(data))

    fig = plt.figure(facecolor="white")
    ax = fig.add_subplot(111)
    plt.rcParams['font.size'] = 20

    rects = ax.bar(ind - width / 2, data, width, color='steelblue')
    
    # add some text for labels, title and axes ticks
    ax.set_ylabel('REBA Score')
    ax.set_xlabel('Timestep')
    ax.set_ylim([0, 6])
    ax.set_xlim([0, len(data)])
    
    plt.plot((start, start), (0, 6), 'k--',lw=2)
    plt.plot((end, end), (0, 6), 'k--',lw=2)

    # save svg file
    fig.set_size_inches(12.8, 10.24)
    plt.savefig(savefile, dpi=100, facecolor=fig.get_facecolor(), transparent=False)
    plt.close()

In [7]:
def extract_reba_value(filename, user, cond, shape):
    group_dict = {}
    if exists(filename):
        with open(filename) as data_file:
            data = json.load(data_file)
        if data['posture_score']['data']:
            start = data['posture_score']['insertion_index'][0]
            end = data['posture_score']['insertion_index'][1]
            
            # plot the figure
            fig_dir = join(rospack.get_path('thr_xp_human_comfort'), res_folder, user, 'figures')
            if not exists(fig_dir):
                os.makedirs(fig_dir)
            cond_dir = join(fig_dir, cond)
            if not exists(cond_dir):
                os.makedirs(cond_dir)
            filename = join(cond_dir, shape + '.svg')
            if exists(filename):
                os.remove(filename)
            
            if start != -1:
                plot_results(data['posture_score']['data'], start, end, filename)

                # calculate the score per group
                for g in groups:
                    score = np.zeros(len(data['posture_score']['data_per_joint'][g[0] + '_0']))
                    for i in range(g[1]):
                        score = score + np.array(data['posture_score']['data_per_joint'][g[0] + '_' + str(i)])
                    group_dict[g[0]] = mean(score[start:end])

                return mean(data['posture_score']['data'][start:end]), group_dict
            else:
                print 'Problem with file ' + filename +', incorrect detection'
                return None, {} 
        else:
            print 'Problem with file ' + filename
            return None, {}
    else:
        print 'File ' + filename + ' not yet calculated'
        return None, {}

In [8]:
def calculate_pose_diff(filename):
    if exists(filename):
        with open(filename) as data_file:
            data = json.load(data_file)
        # analyze the joint difference
        for j in reba_joints:
            if data['distance_to_expected']['reba'][j]:
                dist_dict[j].append(mean(data['distance_to_expected']['reba'][j])) 

In [9]:
directory = join(rospack.get_path("thr_xp_human_comfort"), res_folder)
user_list = [name for name in listdir(directory) if isdir(join(directory, name))]

mean_score = {}
conditions = ['relative', 'fixed', 'reba']
perm_list = list(permutations(conditions, 2))
comb_list = list(combinations(conditions, 2))

groups = [['left_elbow', 1], ['right_elbow', 1], ['left_shoulder', 2], ['right_shoulder', 2],
          ['left_wrist', 2], ['right_wrist', 2], ['spine', 3], ['neck', 3]]
test_parts = ['neck', 'spine', 'right_shoulder', 'right_elbow', 'right_wrist', 'total']

for c in conditions:
    mean_score[c] = {}
    mean_score[c]['total'] = []
    for g in groups:
         mean_score[c][g[0]] = []
            
for c in perm_list:
    mean_score[c[0] + '-' + c[1]] = {}
    mean_score[c[0] + '-' + c[1]]['total'] = [[], []]
    for g in groups:
         mean_score[c[0] + '-' + c[1]][g[0]] = [[], []]

dist_dict = {}
for j in reba_joints:
    dist_dict[j] = []

# loop through all the users
for user in user_list:
    # get dictionnary for results
    dir_results = join(directory, user, 'records')
    conf_file = join(directory, user, 'user_config.json')
    with open(conf_file) as datafile:
        user_config = json.load(datafile)

    if user_config['laterality'] == 'right':
        cond = [name for name in listdir(dir_results) if isdir(join(dir_results, name))]
        shapes = ['star', 'hexagon', 'ellipse', 'slice', 'trapezium']

        for i in range(len(cond)):
            c = cond[i]
            user_res = []
            user_dict = {}
            for g in groups:
                 user_dict[g[0]] = []
            for s in shapes:
                if rospy.is_shutdown():
                    sys.exit(0)
                # import file
                filename = join(dir_results, c, s, 'replay.json')
                score, group_dict = extract_reba_value(filename, user, c, s)

                if score is not None:
                    user_res.append(score)
                    for g in groups:
                        user_dict[g[0]].append(group_dict[g[0]])
                
                # calculate the dist diff
                calculate_pose_diff(filename)
            if user_res:
                m = mean(user_res)
                mean_score[c]['total'].append(m)
                mean_score[cond[0] + '-' + cond[1]]['total'][i].append(m)
                #mean_score[c]['total'] += user_res
            for g in groups:
                if user_dict[g[0]]:
                    m = mean(user_dict[g[0]])
                    mean_score[c][g[0]].append(m)
                    mean_score[cond[0] + '-' + cond[1]][g[0]][i].append(m)
                    #mean_score[c][g[0]] += user_dict[g[0]]

Problem with file /home/buschbapti/catkin_ws/src/thr_infrastructure/thr_experiments/thr_xp_human_comfort/results/3306/figures/fixed/ellipse.svg, incorrect detection
Problem with file /home/buschbapti/catkin_ws/src/thr_infrastructure/thr_experiments/thr_xp_human_comfort/results/17/figures/reba/hexagon.svg, incorrect detection
Problem with file /home/buschbapti/catkin_ws/src/thr_infrastructure/thr_experiments/thr_xp_human_comfort/results/17/figures/relative/star.svg, incorrect detection
Problem with file /home/buschbapti/catkin_ws/src/thr_infrastructure/thr_experiments/thr_xp_human_comfort/results/3734/figures/reba/star.svg, incorrect detection
Problem with file /home/buschbapti/catkin_ws/src/thr_infrastructure/thr_experiments/thr_xp_human_comfort/results/3734/figures/reba/slice.svg, incorrect detection
Problem with file /home/buschbapti/catkin_ws/src/thr_infrastructure/thr_experiments/thr_xp_human_comfort/results/988/figures/fixed/hexagon.svg, incorrect detection
Problem with file /home

In [10]:
plot_reba(mean_score, join(directory, 'reba_score.svg'))

In [11]:
data_test = {}
for c in comb_list:
    data_test[c[0] + '-' + c[1]] = {}
    m_c0 = mean_score[c[0] + '-' + c[1]]['total'][0] + mean_score[c[1] + '-' + c[0]]['total'][1]
    m_c1 = mean_score[c[0] + '-' + c[1]]['total'][1] + mean_score[c[1] + '-' + c[0]]['total'][0]
    data_test[c[0] + '-' + c[1]]['total'] = [m_c0, m_c1]
    for g in groups:
        m_c0 = mean_score[c[0] + '-' + c[1]][g[0]][0] + mean_score[c[1] + '-' + c[0]][g[0]][1]
        m_c1 = mean_score[c[0] + '-' + c[1]][g[0]][1] + mean_score[c[1] + '-' + c[0]][g[0]][0]
        data_test[c[0] + '-' + c[1]][g[0]] = [m_c0, m_c1]

In [12]:
# analyze differences
stat_res = {}
for t in test_parts:
    stat_res[t] = {}
    h, p_o_f = ttest_rel(data_test['fixed-reba'][t][0], data_test['fixed-reba'][t][1])
    stat_res[t]['optimized-fixed'] = p_o_f
    h, p_o_r = ttest_rel(data_test['relative-reba'][t][0], data_test['relative-reba'][t][1])
    stat_res[t]['optimized-relative'] = p_o_r
    h, p_r_f = ttest_rel(data_test['relative-fixed'][t][0], data_test['relative-fixed'][t][1])
    stat_res[t]['relative-fixed'] = p_r_f

In [13]:
for key, values in stat_res.iteritems():
    print key + ': ' + str(values)

right_wrist: {'optimized-fixed': 0.77949131877748012, 'optimized-relative': 0.00084074247451803262, 'relative-fixed': 5.2083777632516809e-07}
spine: {'optimized-fixed': 0.37193945364416581, 'optimized-relative': 0.052904597891623634, 'relative-fixed': 0.19771537017867216}
right_elbow: {'optimized-fixed': 0.2500597596091611, 'optimized-relative': 0.42912807636922423, 'relative-fixed': 0.49982765841472077}
right_shoulder: {'optimized-fixed': 0.00045461826391898324, 'optimized-relative': 0.01038320914645666, 'relative-fixed': 0.015073011141784992}
total: {'optimized-fixed': 0.00016906221112438348, 'optimized-relative': 0.073308898184855323, 'relative-fixed': 0.015925046916258238}
neck: {'optimized-fixed': 0.49328834634270302, 'optimized-relative': 0.1052242505175668, 'relative-fixed': 0.076448065357420755}


In [14]:
total_dist = []
for key, value in dist_dict.iteritems():
    total_dist.append(mean(value))
    print key + ': ' + str(mean(value))
print '------------'

head_mean = mean([dist_dict['neck_0'], dist_dict['neck_1'], dist_dict['neck_2']])
spine_mean = mean([dist_dict['spine_0'], dist_dict['spine_1'], dist_dict['spine_2']])
shoulder_mean = mean([dist_dict['right_shoulder_0'],
                      dist_dict['right_shoulder_1'],
                      dist_dict['right_shoulder_2']])
elbow_mean = mean([dist_dict['right_elbow_0'], dist_dict['right_elbow_1']])
wrist_mean = mean([dist_dict['right_wrist_0'], dist_dict['right_wrist_1']])

print 'head: ' + str(head_mean)
print 'spine: ' + str(spine_mean)
print 'shoulder: ' + str(shoulder_mean)
print 'elbow: ' + str(elbow_mean)
print 'wrist: ' + str(wrist_mean)

print '------------'
print 'total mean: ' + str(mean(total_dist))

neck_2: 0.354319938136
neck_0: 0.452571946815
neck_1: 0.487106416469
right_wrist_0: 0.36446815512
right_wrist_1: 0.47898644948
right_shoulder_0: 0.194662871789
right_shoulder_1: 0.343322730522
right_shoulder_2: 0.588609646012
right_elbow_0: 0.780195847858
right_elbow_1: 0.0651089559725
spine_2: 0.145911054851
spine_1: 0.173810044262
spine_0: 0.102491618142
------------
head: 0.43133276714
spine: 0.140737572418
shoulder: 0.375531749441
elbow: 0.422652401915
wrist: 0.4217273023
------------
total mean: 0.348581975033


In [19]:
print mean(mean_score['fixed']['total'])
print mean(mean_score['relative']['total'])
print mean(mean_score['reba']['total'])
print '------------------------'
print mean(mean_score['relative']['right_wrist'])
print mean(mean_score['reba']['right_wrist'])

3.35548715912
2.63608573374
2.21021939041
------------------------
3.38030202251
2.51514134871
